## Goal:  understanding activity

The term activity in this document referes to actions performed by contributors. Based on the characterization of contributors, we will look at activities of the different identified groups over time. Having into account the the different data sources, we have into account the following actions:

 * Git: sending a commit
 * GitHub issues: opening an issue, closing an issue
 * GitHub pull requests: submiting a pull request, accepting (merging) a pull request
 * Bugzilla: opening a ticket, closing a ticket
 * Mailing Lists: sending a message
 * Discourse: initiating a thread, commmenting in a thread
  
This goal is refined in the following questions:

**Questions**:

For each of the activities and contributor groups identified above,

* How does activity evolve over time?

The metrics identified are:

**Metrics**: 

* Git:
  * Number of commits authored
* GitHub:
  * Number of issues opened
  * Number of issues closed
  * Number of pull requests opened
  * Number of pull requests merged
* Bugzilla:
  * Number of tickets opened
  * Number of tickets closed
* Mailing lists:
  * Number of e-mails sent
* Discourse:
  * Number of threads initiated
  * Number of comments posted
  
These metrics will be computed for the speficied contributor groups, over time.

### Metric Calculations
First we need to load a connection against the proper ES instance. We use an external module to load credentials from a file that will not be shared. If you want to run this, please use your own credentials, just put them in a file named '.settings' (in the same directory as this notebook) following the example file 'settings.sample'.

**TODO**: add bot and merges filtering

In [3]:
import pandas

from util import ESConnection
from elasticsearch_dsl import Search

es_conn = ESConnection()

#### Git: Number of commits authored
Commits are contributions in terms of Git. Looking at them we can measure not only global activity of projects and organizations, but also how these projects and organizations evolve through time.

Global numbers were already provided in [List of projects](http://localhost:8888/notebooks/mozilla-contribution-analysis/data-analysis/Understanding%20Contributors.ipynb#List-of-Projects) and [List of Organizations](http://localhost:8888/notebooks/mozilla-contribution-analysis/data-analysis/Understanding%20Contributors.ipynb#List-of-organizations) sections of the previous goal.

**TODO**: provide plots like the following to show evolution through time:
 * by  project: https://analytics.mozilla.community:443/goto/1dd4de57a16b002661e8d44e43f3e7fa
 * by organization: https://analytics.mozilla.community:443/goto/3945d94db113a378f60893a67c32bbcc

#### GitHub:
  * Number of issues opened
  * Number of issues closed
  * Number of pull requests opened
  * Number of pull requests merged

**TODO**: provide plots like:
  * PRs: https://analytics.mozilla.community:443/goto/99a2cf4d0e06986fe5886ccafa01c88f
  * Issues: https://analytics.mozilla.community:443/goto/db1d0243582548c8f8a0469f3e099677

#### Bugzilla:
  * Number of tickets opened
  * Number of tickets closed
  
**TODO**: provide plots like:
  * Open tickets: https://analytics.mozilla.community:443/goto/4c749e3e1cdada7e3aeaf81ddf25e4fb
  * Closed tickets: https://analytics.mozilla.community:443/goto/2140cf24fb3b876d27b1ba62c21141bc


* Mailing lists:
  * Number of e-mails sent
    * https://analytics.mozilla.community:443/goto/d64333339c487777686182cb62ad1053
  
* Discourse:
  * Number of threads initiated
    * https://analytics.mozilla.community:443/goto/71771202d68a10cc422c6bda86c7cf3e
  * Number of comments posted
    * https://analytics.mozilla.community:443/goto/73c76412902180d14e0418d03fb30884
  
These metrics will be computed for the speficied contributor groups, over time.